In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from TF_data_prepro import*

In [2]:
## load data ##
path=r'df_clean.pkl'
df= pd.read_pickle(path)

In [3]:
## set target ##
pd.set_option('display.max_columns', None)
df['target']=df['price per unit']
X=df.drop(columns='price per unit').values
Y=df['target'].values
df.reset_index(inplace=True,drop=True)
df

,order quantity,Customer,By_way,Types,SPEC_1st_L1,SPEC_1st_L2,SPEC_1st_L3,SPEC_1st_N,Ap_outer_r,Ap_L,AP_w,cen_outter_r,cen_L,cen_material,Shell_outer_r,shell_L,shell_material,SPEC_2nd_L1,SPEC_2nd_L2,SPEC_2nd_L3,target
0,1,新唐,SEMI,國內終端,X,X,S,20.0,105.000000,136.0,2.85,29.000000,136.0,SUS420,105.000000,87.85,SUS304,W,M,R,53000.000
1,1,UMC 8C,SEMI,國內終端,X,X,S,625.0,76.199997,204.0,1.02,44.000000,204.0,SUS420,76.199997,87.50,AL6061,A,M,N,49500.000
2,1,UMC 8C,SEMI,國內終端,X,X,S,625.0,76.199997,204.0,1.02,44.000000,204.0,SUS420,76.199997,87.50,AL6061,A,M,N,49500.000
3,1,AMAT,DISPLAY,國內設備商,X,X,S,30.0,128.000000,180.5,6.48,54.900002,180.5,SUS420,128.000000,121.00,SUS304,W,M,R,59015.000
4,1,UMC 8C,SEMI,國內終端,X,X,S,625.0,76.199997,204.0,1.02,44.000000,204.0,SUS420,76.199997,87.50,AL6061,A,M,N,49500.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,2,AMAT,DISPLAY,國內設備商,A,X,S,40.0,170.000000,241.0,14.33,70.000000,241.0,SUS420,170.000000,127.00,SUS304,C,N,R,80870.200
779,1,AMAT,DISPLAY,國內設備商,A,X,S,40.0,170.000000,241.0,14.33,70.000000,241.0,SUS420,170.000000,127.00,SUS304,C,N,R,80870.200
780,1,AMAT,DISPLAY,國內設備商,A,X,S,40.0,170.000000,241.0,14.33,70.000000,241.0,SUS420,170.000000,127.00,SUS304,C,N,R,81705.615
781,1,AMAT,DISPLAY,國內設備商,A,X,S,40.0,170.000000,241.0,14.33,70.000000,241.0,SUS420,170.000000,127.00,SUS304,C,N,R,80754.150


In [4]:
## Train, Test spliting ##
X_train, test, val=split_data(df, 0.2)

500 train dataset
126 validation dataset
157 test dataset


In [5]:
df,num,cat=feature_col_clean_split(df)

/Users/linli-shang/opt/anaconda3/envs/data/lib/python3.8/site-packages/pandas/core/frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [6]:
num.to_excel('num.xlsx')

In [7]:
cat.describe(include='all')

,Customer,By_way,Types,SPEC_1st_L1,SPEC_1st_L2,SPEC_1st_L3,cen_material,shell_material,SPEC_2nd_L1,SPEC_2nd_L2,SPEC_2nd_L3
count,783,783,783,783,783,783,783,783,783,783,783
unique,75,6,6,4,8,6,5,5,3,4,7
top,KJLC,OTHERS,國內設備商,B,X,S,SUS420,SUS304,C,N,R
freq,220,285,326,439,328,582,687,675,573,676,604


In [8]:
feature_columns = []

In [9]:
## selectr all numerical col ##

for header in num.columns[:-1]:
    feature_columns.append(feature_column.numeric_column(header))

In [10]:
## bucket numerical col ##
SPEC_1st_N = feature_column.numeric_column('SPEC_1st_N')
SPEC_1st_N_buckets = feature_column.bucketized_column(SPEC_1st_N, boundaries=[num['SPEC_1st_N'].unique().sort()])
feature_columns.append(SPEC_1st_N_buckets)

In [11]:
feature_columns

[NumericColumn(key='order quantity', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SPEC_1st_N', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Ap_outer_r', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Ap_L', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='AP_w', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='cen_outter_r', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='cen_L', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Shell_outer_r', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='shell_L', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 BucketizedColumn(source_column=NumericColumn(key='SPEC_1st_N', shape=(1,),

In [12]:
cat.to_excel('cat.xlsx')

In [13]:
for feature_name in cat.columns:
    vocabulary = cat[feature_name].unique()
    cat_c = tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)
    embeding = feature_column.embedding_column(cat_c, dimension=50)
    feature_columns.append(embeding)

## catagorical Col, indicator_col ##
for col_name in cat.columns[1:]:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, cat[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)

By_way_col = feature_column.categorical_column_with_vocabulary_list(
      'By_way', list(cat.By_way.unique()))

In [14]:
feature_columns

[NumericColumn(key='order quantity', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SPEC_1st_N', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Ap_outer_r', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Ap_L', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='AP_w', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='cen_outter_r', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='cen_L', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Shell_outer_r', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='shell_L', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 BucketizedColumn(source_column=NumericColumn(key='SPEC_1st_N', shape=(1,),

# embedding columns
Customer_col = feature_column.categorical_column_with_vocabulary_list(
      'Customer', cat.Customer.unique())
Customer_embedding = feature_column.embedding_column(Customer_col,dimension=16)
feature_columns.append(Customer_embedding)

feature_columns

# crossed columns
Customer_type_feature = feature_column.crossed_column([Customer_col, By_way_col], hash_bucket_size=100)
feature_columns.append(feature_column.indicator_column(Customer_type_feature))

feature_columns

In [15]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [16]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [18]:
model = Sequential()

model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')

In [19]:
model.fit(x=X_train,y=y_train.values,
          validation_data=(X_test,y_test.values),
          batch_size=128,epochs=400)

NameError: name 'X_train' is not defined

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)